In [ ]:
import ast
import copy
import numpy as np
import os

In [ ]:
import cc.atlas as atlas
import cc.cartography as cartography
import cc.publication as publication
import cc.utils as utils
import cc.tex as tex

In [ ]:
import trove

# Parameters

In [ ]:
config_fp = '/Users/zhafen/paper_repos/cgm_modeling_challenge_paper/literature_review/literature_review.trove'
cp = trove.config_parser.ConfigParser( config_fp )
pm = trove.link_params_to_config(
    config_fp,
)

In [ ]:
atlas_dir = pm['root_data_dir']

In [ ]:
topics = {}
search_strs = {}
for variation in cp.variations:
    topics[variation] = ast.literal_eval( cp.get( variation, 'publications' ) )
    try:
        search_strs[variation] = ast.literal_eval( cp.get( variation, 'search_str' ) )
    except:
        continue

## Load

### Large Atlas

In [ ]:
# Larger atlas
a = atlas.Atlas( atlas_dir, load_bibtex=False, )

In [ ]:
# Vectorization
vp_dict = a.vectorize( overwrite=True )
c = cartography.Cartographer( **vp_dict )

# Search

In [ ]:
print( 'Expanding for topics...' )
for i in range( 3 ):
    print( '\n================================================================\nIteration {}'.format( i ) )
    broken = []
    for topic, pubs_i in topics.items():
        print( '\n================================================================\n{}'.format( topic ) )
        for p_i in pubs_i:
            try:
                a = c.expand( a, center=p_i, n_sources_max=pm['kernel_size'] )
                a.process_abstracts()
            except ( ValueError, AssertionError ) as e:
                if isinstance( e, ValueError ):
                    broken.append( p_i )

        if topic in search_strs:

            # Search string
            print( 'Searching for {}'.format( search_strs[topic] ) )
            p = publication.UnofficialPublication( topic )
            p.process_abstract( abstract_str=search_strs[topic] )
            a.data[topic] = p

        vp_dict = a.vectorize( overwrite=True, projection_fp='pass' )
        c = cartography.Cartographer( **vp_dict )
        try:
            a = c.expand( a, center=p_i, n_sources_max=pm['kernel_size'] )
            a.process_abstracts()
        except ( ValueError, AssertionError ) as e:
            print( 'Search complete for search_str' )

In [ ]:
a.save_data()